In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
!cp drive/MyDrive/emnist_model/pytorch_train_model.py .
!cp drive/MyDrive/emnist_model/pytorch_eval_model.py .
!cp drive/MyDrive/emnist_model/pytorch_model_class.py .
!cp drive/MyDrive/emnist_model/data_loader.py .

In [3]:
from pytorch_train_model import DEVICE

100%|██████████| 561753746/561753746 [00:05<00:00, 108038268.89it/s]


Extracting ./data/EMNIST/raw/gzip.zip to ./data/EMNIST/raw


In [4]:
DEVICE

device(type='cuda')

In [7]:
import pytorch_model_class
char_model = pytorch_model_class.CNN_SRM().to(DEVICE)
char_model.load_model(path = 'drive/MyDrive/emnist_model/saved_models')

created model:  CNN_SRM_letter


In [9]:
import data_loader
train_data, validation_data = data_loader.load_emnist_data(train_noise = False, validation_noise = False)
train_loader, validation_loader = data_loader.create_data_loader(train_data, validation_data)

In [24]:
import torch
validation_correct = 0
char_map = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I',
    9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R',
    18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'
}
for x, y in validation_loader:
  x = x.to(DEVICE)
  y = torch.sub(y, 1)
  y = y.to(DEVICE)
  z = char_model(x)
  _, label = torch.max(z, 1)
  for k in range(len(label)):
    t = z[k].tolist()
    if label[k] != y[k]:
      print(f"WRONG:  pred: {char_map[label[k].item()]}, prob: {t[label[k]]}, correct: {char_map[y[k].item()]}, prob: {t[y[k]]}")
  validation_correct += (label == y).sum().item()

WRONG:  pred: T, prob: 0.7195656299591064, correct: R, prob: 0.2804092764854431
WRONG:  pred: G, prob: 0.9080672264099121, correct: Q, prob: 0.09113471955060959
WRONG:  pred: G, prob: 0.9810864329338074, correct: Q, prob: 0.018913550302386284
WRONG:  pred: I, prob: 0.9488531947135925, correct: L, prob: 0.05114684998989105
WRONG:  pred: G, prob: 0.9999445676803589, correct: F, prob: 6.475389557891731e-09
WRONG:  pred: V, prob: 0.8922040462493896, correct: P, prob: 0.1077871322631836
WRONG:  pred: A, prob: 0.9681249260902405, correct: N, prob: 0.03186357766389847
WRONG:  pred: Q, prob: 0.9999992847442627, correct: G, prob: 7.36527283606847e-07
WRONG:  pred: I, prob: 1.0, correct: L, prob: 3.748788501845013e-10
WRONG:  pred: U, prob: 0.8222949504852295, correct: O, prob: 0.17480307817459106
WRONG:  pred: I, prob: 1.0, correct: J, prob: 2.777851004776011e-10
WRONG:  pred: I, prob: 1.0, correct: L, prob: 5.590992913084847e-08
WRONG:  pred: C, prob: 0.7003138661384583, correct: E, prob: 0.29

In [26]:
for x, y in validation_loader:
  x = x.to(DEVICE)
  y = torch.sub(y, 1)
  y = y.to(DEVICE)
  z = char_model(x)
  _, label = torch.max(z, 1)
  for k in range(len(label)):
    t = z[k].tolist()
    if label[k] == y[k]:
      print(f"CORRECT: tensor: {sorted(t[-3:])}")
  break
  validation_correct += (label == y).sum().item()

CORRECT: tensor: [1.2789843271545789e-24, 1.342607535304742e-21, 4.425823250020074e-17]
CORRECT: tensor: [2.5212622590361838e-21, 6.486748378407373e-17, 2.129604306756131e-14]
CORRECT: tensor: [8.402323162299093e-25, 9.407865772629058e-16, 5.747253225359041e-15]
CORRECT: tensor: [2.207966068881157e-31, 2.851564616468796e-28, 2.0311422954688253e-24]
CORRECT: tensor: [6.169050839870262e-24, 3.8906944691605314e-18, 1.3624347079666498e-14]
CORRECT: tensor: [1.1228897771544648e-23, 4.652032160074921e-22, 1.5219193298631762e-21]
CORRECT: tensor: [3.2389680428433066e-30, 1.820970524000054e-29, 4.604269010679418e-25]
CORRECT: tensor: [3.0475349321904264e-32, 3.9863572064407543e-29, 4.879345319664385e-29]
CORRECT: tensor: [1.8335707012496387e-24, 8.14649957183843e-21, 8.605035663243697e-15]
CORRECT: tensor: [9.893098449400887e-27, 4.509839881439509e-19, 4.215855859232676e-18]
CORRECT: tensor: [3.833702943482518e-28, 4.355411498251937e-25, 4.725755266412877e-24]
CORRECT: tensor: [4.1542365846329

In [70]:
import numpy as np

corrects = [0]*26
totals = [0]*26
errors = np.zeros(shape = (26,26))

for x, y in validation_loader:
  x = x.to(DEVICE)
  y = torch.sub(y, 1)
  y = y.to(DEVICE)
  z = char_model(x)
  _, label = torch.max(z, 1)
  for k in range(len(label)):
    t = z[k].tolist()
    if label[k] != y[k]:
      errors[y[k], label[k]] += 1
    else:
      corrects[y[k]] += 1
    totals[y[k]] += 1

accuracies = [corrects[i]*100/totals[i] for i in range(26)]

for i in range(26):
  errors[i,:] = np.round(errors[i,:]*100/totals[i], 2)
  s = f"When it is {char_map[i]}, model predicts: correct {accuracies[i]}%"
  n = 0
  for j in range(26):
    if errors[i,j] > 1:
      s += f", {char_map[j]} {errors[i,j]}%"
    else:
      n += errors[i,j]
  s += f", others {round(n, 2)}%"
  print(s)

When it is A, model predicts: correct 94.875%, others 5.1%
When it is B, model predicts: correct 97.125%, others 2.85%
When it is C, model predicts: correct 97.25%, E 1.38%, others 1.36%
When it is D, model predicts: correct 95.5%, O 2.88%, others 1.61%
When it is E, model predicts: correct 97.125%, C 1.12%, others 1.72%
When it is F, model predicts: correct 96.25%, T 1.75%, others 1.98%
When it is G, model predicts: correct 84.25%, A 1.75%, Q 11.12%, others 2.84%
When it is H, model predicts: correct 95.75%, N 1.38%, others 2.84%
When it is I, model predicts: correct 75.625%, L 23.12%, others 1.21%
When it is J, model predicts: correct 93.875%, I 3.38%, others 2.73%
When it is K, model predicts: correct 97.625%, others 2.36%
When it is L, model predicts: correct 76.375%, I 21.12%, others 2.49%
When it is M, model predicts: correct 98.625%, others 1.37%
When it is N, model predicts: correct 97.0%, others 2.97%
When it is O, model predicts: correct 97.75%, D 1.25%, others 1.0%
When it i

In [69]:
import numpy as np

corrects = [0]*26
totals = [0]*26
errors = np.zeros(shape = (26,26))

for x, y in validation_loader:
  x = x.to(DEVICE)
  y = torch.sub(y, 1)
  y = y.to(DEVICE)
  z = char_model(x)
  _, label = torch.max(z, 1)
  for k in range(len(label)):
    t = z[k].tolist()
    if label[k] != y[k]:
      errors[label[k], y[k]] += 1
    else:
      corrects[label[k]] += 1
    totals[label[k]] += 1

accuracies = [corrects[i]*100/totals[i] for i in range(26)]

for i in range(26):
  errors[i,:] = np.round(errors[i,:]*100/totals[i], 2)
  s = f"When model predicts {char_map[i]}, it is: correct {accuracies[i]}%"
  n = 0
  for j in range(26):
    if errors[i,j] > 1:
      s += f", {char_map[j]} {errors[i,j]}%"
    else:
      n += errors[i,j]
  s += f", others {round(n, 2)}%"
  print(s)

When model predicts A, it is: correct 93.8195302843016%, G 1.73%, Q 1.85%, others 2.57%
When model predicts B, it is: correct 98.35443037974683%, others 1.67%
When model predicts C, it is: correct 96.40644361833952%, E 1.12%, others 2.47%
When model predicts D, it is: correct 96.1006289308176%, O 1.26%, others 2.67%
When model predicts E, it is: correct 97.00374531835206%, C 1.37%, others 1.6%
When model predicts F, it is: correct 98.84467265725289%, others 1.17%
When model predicts G, it is: correct 85.53299492385787%, A 1.02%, Q 9.9%, others 3.55%
When model predicts H, it is: correct 95.86983729662077%, others 4.16%
When model predicts I, it is: correct 75.06203473945409%, J 3.35%, L 20.97%, others 0.6%
When model predicts J, it is: correct 97.02842377260981%, others 2.99%
When model predicts K, it is: correct 97.99247176913426%, others 2.03%
When model predicts L, it is: correct 74.60317460317461%, I 22.59%, others 2.79%
When model predicts M, it is: correct 98.74843554443054%, oth